In [705]:
import numpy as np
import cv2
#np.random.seed(2)   # block the random

# Definition of the camera parameters
  # focal length
fx = 800
fy = 800
  # center
cx = 0
cy = 0

A = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]]) # intraseca matrix of the camera (3*3)

# Generation of 3 random angles to define the rotation
rax,pitch,yaw = np.random.uniform(0,np.pi,size=3)

# Generation of the rotation matrix

'''Rx = np.array([[1,0,0],[0, np.cos(rax), -np.sin(rax)],[0, np.sin(rax), np.cos(rax)]])
Ry = np.array([[np.cos(pitch), 0, np.sin(pitch)],[0,1,0],[-np.sin(pitch),0, np.cos(pitch)]])
Rz = np.array([[np.cos(yaw), -np.sin(yaw),0],[np.sin(yaw),np.cos(yaw),0],[0,0,1]])

R = Rz @ Ry @ Rx  # rotation matrix  (3*3)'''
R = np.array([[1, 0, 0],[0, -1, 0], [0, 0, -1]])  # with the values of the Kneip's article  (3*3)
print("R=",R,"\n")

# Generation of the camera translation
#T = np.random.uniform(-500,500,size=3)     # T = [tx,ty,tz]  (1*3)
T = np.array([0,0,6])                       # with the values of the Kneip's article
T = T.reshape((3,1))                       # (3*1)

print("T=",T,"\n")

R= [[ 1  0  0]
 [ 0 -1  0]
 [ 0  0 -1]] 

T= [[0]
 [0]
 [6]] 



In [706]:
# Projection from a 3D point to a 2D

def projection3D2D(point3D,T,R,A) :
  # 3D point = [ Xw, Yw, Zw ]'   (1*3)
  # T : camera translation matrix : (3*1)
  # R : camera rotation matrix : (3*3)
  # A : intraseca matrix of the camera : (3*3)

  PI = np.concatenate((np.eye(3),np.zeros((3,1))),axis=1)  # (3*4)

  Rt = np.concatenate((R,T),axis=1)               # (3*4)
  Rt = np.concatenate((Rt,np.array([[0,0,0,1]])),axis=0)   # (4*4)

  point3D_bis = np.concatenate((np.reshape(point3D,(3,1)),np.array([[1]])),axis=0)   #(4*1)

  point2D = A @ PI @ Rt @ point3D_bis   # 2D point = [u, v, w] (3*1)
  point2D = point2D / point2D[2]        # 2D point = [u, v, 1] (3*1)
  return point2D[:2]

'''pt3D = [[3],[1],[4]]
print("point 3D :\n ", pt3D)
pt2D = projection3D2D(pt3D,T,R,A)
print("point 2D :\n", pt2D)'''

'pt3D = [[3],[1],[4]]\nprint("point 3D :\n ", pt3D)\npt2D = projection3D2D(pt3D,T,R,A)\nprint("point 2D :\n", pt2D)'

In [707]:
# test of collinearity
def areCollineary(P1,P2,P3) :
  # return true if P1, P2 and P3 are collineary
  return np.linalg.norm(crossproduct(P1,P2,P3))==0

def crossproduct(P1,P2,P3) :
  v1 = P2 - P1
  v2 = P3 - P1
  return np.cross(v1,v2)


In [708]:
# Creation of 2D, 3D point couples
def point3Daleatoire(x) :
  return np.array([[np.random.uniform(-x,x),np.random.uniform(-x,x),np.random.uniform(-x,x)]])

# 3D points
  # random :
P1 = point3Daleatoire(2)     # (1*3) -> pour P3P
P2 = point3Daleatoire(2)
P3 = point3Daleatoire(2)
P4 = point3Daleatoire(2)
  # set
'''P1 = np.array([[-150,150,1]],dtype=np.float32)
P2 = np.array([[162,-162,1]],dtype=np.float32)
P3 = np.array([[-162,-162,1]],dtype=np.float32)
P4 = np.array([[162,162,1]],dtype=np.float32)'''


points3D = np.concatenate((P1,P2,P3),axis=0);     # (3*3)
print("pt3D\n",points3D,"\n")
print("Collinear points : ", areCollineary(P1,P2,P3))
print("v1 x v2 = ",crossproduct(P1,P2,P3),"\n")

# 2D points
p1 = projection3D2D(P1,T,R,A)   # (2*1)
p1 = np.reshape(p1,(1,2))       # (1,2)
p2 = projection3D2D(P2,T,R,A)
p2 = np.reshape(p2,(1,2))
p3 = projection3D2D(P3,T,R,A)
p3 = np.reshape(p3,(1,2))
p4 = projection3D2D(P4,T,R,A)
p4 = np.reshape(p4,(1,2))


points2D = np.concatenate((p1,p2,p3),axis=0);    # (3*2)
print("pt2D\n",points2D)

# Application of solveP3P to find the camera position
retval, rvec, tvecs =  cv2.solveP3P(points3D,points2D,A,None, flags = cv2.SOLVEPNP_P3P)

rvec = np.array(rvec)
tvecs = np.array(tvecs)


pt3D
 [[ 1.85798282  1.88030571 -0.57289991]
 [-1.83291484  1.19949785 -0.87736158]
 [-1.79270701 -1.02189872 -1.39095766]] 

Collinear points :  False
v1 x v2 =  [[-0.32666984 -1.90787232  8.22632121]] 

pt2D
 [[ 226.13858052 -228.85554109]
 [-213.2113973  -139.53000269]
 [-194.04327226  110.61069688]]


In [709]:
# Calculation of the estimation error :
def distance(pt, pt_estimation):
    erreur = 0
    for i in range(len(pt)):
      erreur += (pt[i] - pt_estimation[i])**2
    return np.sqrt(erreur)

In [710]:


# Measurement of the estimation error
nb_solutions = len(rvec)
print("nombre de solutions trouvées =", nb_solutions)

erreurs = []

for i in range (nb_solutions):
  print("\nSolution ",i+1, ":\n")

  # Recovery of rotation and translation matrices

  rodriguez = rvec[i]     # (3*1)
  R_P3P = cv2.Rodrigues(rodriguez)[0]    # rotation matrix : (3*3)
  print("R_P3P=",R_P3P,"\n")

  T_P3P = tvecs[i]    # translation matrix : (1*3)
  T_P3P = np.reshape(T_P3P,(3,1))    # (3*1)
  print("T=",T_P3P,"\n")


  # Point estimation
  p1_P3P = np.reshape(projection3D2D(P1,T_P3P,R_P3P,A),(1,2))

  p2_P3P = np.reshape(projection3D2D(P2,T_P3P,R_P3P,A),(1,2))
  p3_P3P = np.reshape(projection3D2D(P3,T_P3P,R_P3P,A),(1,2))
  p4_P3P = np.reshape(projection3D2D(P4,T_P3P,R_P3P,A),(1,2))

  # Creation of a table of estimated points
  pt_2D_P3P = np.concatenate((p1_P3P,p2_P3P,p3_P3P,p4_P3P),axis=0)    # (4,2)
  print("pt_2D_P3P = ",pt_2D_P3P,"\n")

  # Creation of a table of original points

  pt_2D = np.concatenate((p1,p2,p3,p4),axis=0)    # (4,2)
  print("pt_2D = ",pt_2D)

  # Calculation of the estimation error
  erreurs.append([0])
  for j in range(len(pt_2D)):
    erreur_pt = distance(pt_2D[j],pt_2D_P3P[j])
    print("erreur = ",erreur_pt)
    erreurs[i]+=erreur_pt

# Fin de best solution
if nb_solutions > 0 :
  indice_max = 0
  max = erreurs[0]
  for i in range(1,len(erreurs)) :
    if erreurs[i]>max :
      max = erreurs[i]
      indice_max = i

  rodriguez = rvec[indice_max]
  R_estime = cv2.Rodrigues(rodriguez)[0]
  T_estime = tvecs[indice_max]    # translation matrix : (1*3)
  T_estime = np.reshape(T_P3P,(3,1))

  print("\n R estimé = ", R_estime,"\n")
  print("T estimé = ", T_estime, "\n")



nombre de solutions trouvées = 4

Solution  1 :

R_P3P= [[ 0.95714447 -0.01610872  0.28916254]
 [-0.0710974  -0.98096729  0.18068851]
 [ 0.28074834 -0.19350371 -0.9400727 ]] 

T= [[0.36040738]
 [0.11397561]
 [6.17665921]] 

pt_2D_P3P =  [[ 226.13858052 -228.85554109]
 [-213.2113973  -139.53000269]
 [-194.04327226  110.61069688]
 [ 301.62683772  185.28383877]] 

pt_2D =  [[ 226.13858052 -228.85554109]
 [-213.2113973  -139.53000269]
 [-194.04327226  110.61069688]
 [ 262.2151603   195.14835541]]
erreur =  2.4780210253313977e-11
erreur =  1.013070333741668e-10
erreur =  6.288074549046965e-11
erreur =  40.627441531830364

Solution  2 :

R_P3P= [[ 0.86906551  0.11787482 -0.4804484 ]
 [ 0.13736022 -0.99050616  0.00545177]
 [-0.47524447 -0.07073244 -0.87700605]] 

T= [[-0.77843342]
 [ 0.26118245]
 [ 5.22983383]] 

pt_2D_P3P =  [[ 226.13858052 -228.85554109]
 [-213.2113973  -139.53000269]
 [-194.04327226  110.61069688]
 [   5.14679385  371.48989439]] 

pt_2D =  [[ 226.13858052 -228.85554109]
 [